# <font color="#33ccff">Notebook for downloading and scoring tweets that match given hashtags</font>
<img src=https://www.softwareadvice.com/resources/wp-content/uploads/The-Best-Free-Tools-for-Twitter-Sentiment-Analysis-Tile.png>

### Set credentials

In [11]:
consumer_key = "consumer_key"
consumer_secret = "consumer_secret"
access_token_key = "access_token_key"
access_token_secret = "asccess_token_secret"

credentials = dict(
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token_key=access_token_key,
    access_token_secret=access_token_secret
)

### Save credentials to file

In [12]:
# import pickle

# filename = "credentials"
# with open(filename, "wb") as cred_fd:
#     pickle.dump(credentials, cred_fd)

### Load credentials 
<font color='red'>NOTE:</font> that you don't want other people to know them, so do not commit and push accidently do github or something

In [13]:
import pickle

with open("credentials", "rb") as cred_fd:
    credentials = pickle.load(cred_fd)

In [20]:
import logging
import os

logging.basicConfig(level=logging.INFO)


In [38]:
import twitter


class TwitterDownloader:
    def __init__(self, credentials):
        self.api = twitter.Api(**credentials, sleep_on_rate_limit=True)
        self.data = []
      
    @staticmethod
    def extract_tweet_data(tweet):
        return {
            "created_at" : tweet.created_at,
            "created_at_seconds" : tweet.created_at_in_seconds,
            "id" : tweet.id_str,
            "lang" : tweet.lang,
            "user" : tweet.user.name,
            "full_text" : tweet.full_text
        }
    
    def tweets_with_hashtags_generator(self, hashtags, lang, max_tweets=1000):
        """
        Download tweets with given hashtags. Minimum is 100.
        :param hashtags: list of lists, if you want tweets with multiple hashtags inside or string (or 1 elem list) if 
                         only one hashtag in tweet.
        """
        downloaded_tweets = 0
        previous_downloaded = -1
        max_id = 0
        
        while downloaded_tweets < max_tweets:
            if previous_downloaded == downloaded_tweets:
                print("No more tweets to download, ending...")
                break
                
                
            previous_downloaded = downloaded_tweets
            for in_tweet_hashtags in hashtags:
                
                if isinstance(in_tweet_hashtags, str):
                    hashtags_query = "%23" + in_tweet_hashtags
                elif isinstance(in_tweet_hashtags, list) or isinstance(in_tweet_hashtags, tuple):
                    hashtags_query = "%23" + in_tweet_hashtags[0] if len(in_tweet_hashtags) == 1 else "%23" + "%23".join(in_tweet_hashtags)
                else:
                    raise(
                        TypeError(
                            "Elements of hashtags list have to be list or tuple - if multiple hashtags or string if 1 wanted"
                        )
                    )
                
                # This is next chunk of tweets so we want to download tweets older than the oldest already downloaded
                # Initialized by 0
                
                # TODO: Test this 
                if max_id ==0:
                    query = 'q={}&lang={}&tweet_mode=extended&count=100'.format(hashtags_query,lang)
                else:
                    query = 'q={}&lang={}&tweet_mode=extended&count=100&max_id={}'.format(hashtags_query,lang,max_id - 1)
                
                logging.info("Query: {}".format(hashtags_query))
                
                tweets = api.GetSearch(
                        raw_query=query
                    )
                if len(tweets) == 0:
                    print("No more tweets for hashtag:", in_tweet_hashtags, sep=" ")
                    break
                    
                for tweet in tweets:
                    parsed_tweet = self.extract_tweet_data(tweet)
                    self.data.append(parsed_tweet)
                    if max_id ==0:
                        max_id = parsed_tweet["id"]
                    else:
                        max_id = min(max_id, parsed_tweet["id"])
                    yield parsed_tweet
                    downloaded_tweets+=1
    

### Create downloader object with credentials

In [39]:
td = TwitterDownloader(credentials)

### Generate hashtags to download tweets from

In [40]:
import itertools

hashtags1 = ["android", "ios", "smartfon", "telefon"]
hashtags2 = ["apple", "iphone", "xiaomi", "htc" or "samsung"]

hashtags = [list(element) for element in itertools.product(hashtags1, hashtags2)]

hashtags

[['android', 'apple'],
 ['android', 'iphone'],
 ['android', 'xiaomi'],
 ['android', 'htc'],
 ['ios', 'apple'],
 ['ios', 'iphone'],
 ['ios', 'xiaomi'],
 ['ios', 'htc'],
 ['smartfon', 'apple'],
 ['smartfon', 'iphone'],
 ['smartfon', 'xiaomi'],
 ['smartfon', 'htc'],
 ['telefon', 'apple'],
 ['telefon', 'iphone'],
 ['telefon', 'xiaomi'],
 ['telefon', 'htc']]

### Start downloading tweets, rate limits will be held automatically

In [41]:
tweets_generator = td.tweets_with_hashtags_generator(hashtags=hashtags,lang="pl", max_tweets=2000)

In [42]:
# remove tweets_file if you want to start from scratch
os.remove(csv_file)

FileNotFoundError: [Errno 2] No such file or directory: 'tweets.csv'

In [43]:
import os
import pandas as pd

csv_file = "tweets.csv"

if os.path.isfile(csv_file):
    print("Loading from file:", csv_file, sep=" ")
    df = pd.read_csv(csv_file)
    df.reindex()
else:
    df = pd.DataFrame()

for tweet in tweets_generator:
    continue
    tweet["score"] = None
    df = df.append(tweet, ignore_index=True)
    df.to_csv(path_or_buf=csv_file, index=False)


INFO:root:Query: %23android%23apple
INFO:root:Query: %23android%23iphone
INFO:root:Query: %23android%23xiaomi
INFO:root:Query: %23android%23htc
INFO:root:Query: %23ios%23apple
INFO:root:Query: %23ios%23iphone
INFO:root:Query: %23ios%23xiaomi
INFO:root:Query: %23ios%23htc
INFO:root:Query: %23smartfon%23apple
INFO:root:Query: %23smartfon%23iphone
INFO:root:Query: %23smartfon%23xiaomi
INFO:root:Query: %23smartfon%23htc
INFO:root:Query: %23telefon%23apple
INFO:root:Query: %23telefon%23iphone
INFO:root:Query: %23telefon%23xiaomi
INFO:root:Query: %23telefon%23htc


No more tweets to download, ending...


In [ ]:
df

In [ ]:
("android" or "ios" or "smartfon" or "telefon") and \
("apple" or "iphone" or "xiaomi" or "htc" or "samsung") in ["andriod","iphone"]

## Interface for easy <font color="#33ccff">tweets </font> scoring

In [ ]:
# Load the most recent version of data
df = pd.read_csv(csv_file)
df

In [ ]:
import math
from IPython.display import clear_output

for i, tweet in df.iterrows():
    score = tweet.score
    if math.isnan(score) or score > 1.0 or score <-1.0:
        score = 45444 # random, not between [-1,1]
    else:
        continue
        
    while score > 1.0 or score <-1.0:
        tmp_score = input("{}: {}\n".format(i, tweet.full_text))
        try:
            score = float(tmp_score)
            if score <-1.0 or score > 1.0:
                raise ValueError
        except ValueError:
            print("Score must be betweet -1.0 and 1.0")
    clear_output()
    
    df.loc[i, "score"] = score
    
    # save scored data
    df.to_csv(path_or_buf=csv_file, index=False)